In [ ]:
import pandas as pd
from eolearn.core import EOPatch, FeatureType, OverwritePermission
from fs_s3fs import S3FS
from sentinelhub import SHConfig
from sg_utils.processing import multiprocess

In [ ]:
AWS_ACCESS_KEY_ID = ''
AWS_SECRET_ACCESS_KEY = ''

In [ ]:
filesystem = S3FS(bucket_name='',
                  aws_access_key_id=AWS_ACCESS_KEY_ID,
                  aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                  region='eu-central-1')

In [ ]:
config = SHConfig()
config.aws_access_key_id = AWS_ACCESS_KEY_ID
config.aws_secret_access_key = AWS_SECRET_ACCESS_KEY

In [ ]:
metadata_ms4 = pd.read_parquet(filesystem.openbin('metadata/deimos_ms4_metadata.pq'))
metadata_pan = pd.read_parquet(filesystem.openbin('metadata/deimos_pan_metadata.pq'))

metadata_ms4.START_TIME = pd.to_datetime(metadata_ms4.START_TIME)

In [ ]:
DEIMOS_DIR = ''

In [ ]:
metadata_pan.START_TIME = pd.to_datetime(metadata_pan.START_TIME)

In [ ]:
def add_new_metadata(eop_path):
    eop = EOPatch.load(f'{DEIMOS_DIR}/{eop_path}', filesystem=filesystem, lazy_loading=True)
    meta_info = eop.meta_info
    for timestamp in meta_info['metadata']:
        stats = metadata_ms4[metadata_ms4.START_TIME == timestamp]
        stats_pan = metadata_pan[metadata_pan.START_TIME == timestamp]
        # Add "cloudy" median
        meta_info['metadata'][timestamp]['MS4']['CLM_BAND_STATS_'] = {}
        meta_info['metadata'][timestamp]['PAN']['CLM_BAND_STATS'] = {}

        for i in range(0, 4):
            meta_info['metadata'][timestamp]['MS4']['BAND_STATS'][f'STX_MEDIAN_{i+1}'] = stats[f'STX_MEDIAN_{i+1}'].iloc[0]
            meta_info['metadata'][timestamp]['MS4']['CLM_BAND_STATS'][
                f'STX_CLM_MEDIAN_{i+1}'] = stats[f'STX_CLM_MEDIAN_{i+1}'].iloc[0]
            meta_info['metadata'][timestamp]['MS4']['CLM_BAND_STATS'][
                f'STX_CLM_MEAN_{i+1}'] = stats[f'STX_CLM_MEAN_{i+1}'].iloc[0]
            meta_info['metadata'][timestamp]['MS4']['CLM_BAND_STATS'][
                f'STX_CLM_STDV_{i+1}'] = stats[f'STX_CLM_STDV_{i+1}'].iloc[0]

        meta_info['metadata'][timestamp]['PAN']['BAND_STATS'][f'STX_MEDIAN_1'] = stats_pan[f'STX_MEDIAN_1'].iloc[0]
        meta_info['metadata'][timestamp]['PAN']['CLM_BAND_STATS'][f'STX_CLM_MEDIAN_1'] = stats_pan[f'STX_CLM_MEDIAN_1'].iloc[0]
        meta_info['metadata'][timestamp]['PAN']['CLM_BAND_STATS'][f'STX_CLM_MEAN_1'] = stats_pan[f'STX_CLM_MEAN_1'].iloc[0]
        meta_info['metadata'][timestamp]['PAN']['CLM_BAND_STATS'][f'STX_CLM_STDV_1'] = stats_pan[f'STX_CLM_STDV_1'].iloc[0]

    eop.save(path=f'{DEIMOS_DIR}/{eop_path}', filesystem=filesystem,
             features=[FeatureType.META_INFO], overwrite_permission=OverwritePermission.OVERWRITE_FEATURES)


def add_new_metadata_pansharpened(eop_path):
    eop = EOPatch.load(f'{DEIMOS_DIR}/{eop_path}', filesystem=filesystem, lazy_loading=True)
    meta_info = eop.meta_info
    for timestamp in meta_info['metadata']:
        stats = metadata_ms4[metadata_ms4.START_TIME == timestamp]
        stats_pan = metadata_pan[metadata_pan.START_TIME == timestamp]
        # Add "cloudy" median
        meta_info['metadata'][timestamp]['MS4']['CLM_BAND_STATS_PANSHARPENED'] = {}

        for i in range(0, 4):
            meta_info['metadata'][timestamp]['MS4']['CLM_BAND_STATS_PANSHARPENED'][
                f'STX_CLM_MEDIAN_PANSHARPENED_{i+1}'] = stats[f'STX_CLM_MEDIAN_PANSHARPENED_{i+1}'].iloc[0]
            meta_info['metadata'][timestamp]['MS4']['CLM_BAND_STATS_PANSHARPENED'][
                f'STX_CLM_MEAN_PANSHARPENED_{i+1}'] = stats[f'STX_CLM_MEAN_PANSHARPENED_{i+1}'].iloc[0]
            meta_info['metadata'][timestamp]['MS4']['CLM_BAND_STATS_PANSHARPENED'][
                f'STX_CLM_STDV_PANSHARPENED_{i+1}'] = stats[f'STX_CLM_STDV_PANSHARPENED_{i+1}'].iloc[0]

    eop.save(path=f'{DEIMOS_DIR}/{eop_path}', filesystem=filesystem,
             features=[FeatureType.META_INFO], overwrite_permission=OverwritePermission.OVERWRITE_FEATURES)


def add_new_metadata_radiance(eop_path):
    eop = EOPatch.load(f'{DEIMOS_DIR}/{eop_path}', filesystem=filesystem, lazy_loading=True)
    meta_info = eop.meta_info
    for timestamp in meta_info['metadata']:
        stats = metadata_ms4[metadata_ms4.START_TIME == timestamp]
        stats_pan = metadata_pan[metadata_pan.START_TIME == timestamp]
        # Add "cloudy" median
        meta_info['metadata'][timestamp]['MS4']['CLM_RADIANCE_BAND_STATS'] = {}
        meta_info['metadata'][timestamp]['PAN']['CLM_RADIANCE_BAND_STATS'] = {}

        for i in range(0, 4):
            meta_info['metadata'][timestamp]['MS4']['CLM_RADIANCE_BAND_STATS'][
                f'STX_CLM_RADIANCE_MEDIAN_{i+1}'] = stats[f'STX_CLM_RADIANCE_MEDIAN_{i+1}'].iloc[0]
            meta_info['metadata'][timestamp]['MS4']['CLM_RADIANCE_BAND_STATS'][
                f'STX_CLM_RADIANCE_MEAN_{i+1}'] = stats[f'STX_CLM_RADIANCE_MEAN_{i+1}'].iloc[0]
            meta_info['metadata'][timestamp]['MS4']['CLM_RADIANCE_BAND_STATS'][
                f'STX_CLM_RADIANCE_STDV_{i+1}'] = stats[f'STX_CLM_RADIANCE_STDV_{i+1}'].iloc[0]

        meta_info['metadata'][timestamp]['PAN']['CLM_RADIANCE_BAND_STATS'][f'STX_CLM_RADIANCE_MEDIAN_1'] = stats_pan[f'STX_CLM_RADIANCE_MEDIAN_1'].iloc[0]
        meta_info['metadata'][timestamp]['PAN']['CLM_RADIANCE_BAND_STATS'][f'STX_CLM_RADIANCE_MEAN_1'] = stats_pan[f'STX_CLM_RADIANCE_MEAN_1'].iloc[0]
        meta_info['metadata'][timestamp]['PAN']['CLM_RADIANCE_BAND_STATS'][f'STX_CLM_RADIANCE_STDV_1'] = stats_pan[f'STX_CLM_RADIANCE_STDV_1'].iloc[0]

    eop.save(path=f'{DEIMOS_DIR}/{eop_path}', filesystem=filesystem,
             features=[FeatureType.META_INFO], overwrite_permission=OverwritePermission.OVERWRITE_FEATURES)


def add_new_metadata_radiance_pansharpened(eop_path):
    eop = EOPatch.load(f'{DEIMOS_DIR}/{eop_path}', filesystem=filesystem, lazy_loading=True)
    meta_info = eop.meta_info
    for timestamp in meta_info['metadata']:
        stats = metadata_ms4[metadata_ms4.START_TIME == timestamp]
        stats_pan = metadata_pan[metadata_pan.START_TIME == timestamp]
        # Add "cloudy" median
        meta_info['metadata'][timestamp]['MS4']['CLM_RADIANCE_BAND_STATS_PANSHARPENED'] = {}

        for i in range(0, 4):
            meta_info['metadata'][timestamp]['MS4']['CLM_RADIANCE_BAND_STATS_PANSHARPENED'][
                f'STX_CLM_RADIANCE_MEDIAN_PANSHARPENED_{i+1}'] = stats[f'STX_CLM_RADIANCE_MEDIAN_PANSHARPENED_{i+1}'].iloc[0]
            meta_info['metadata'][timestamp]['MS4']['CLM_RADIANCE_BAND_STATS_PANSHARPENED'][
                f'STX_CLM_RADIANCE_MEAN_PANSHARPENED_{i+1}'] = stats[f'STX_CLM_RADIANCE_MEAN_PANSHARPENED_{i+1}'].iloc[0]
            meta_info['metadata'][timestamp]['MS4']['CLM_RADIANCE_BAND_STATS_PANSHARPENED'][
                f'STX_CLM_RADIANCE_STDV_PANSHARPENED_{i+1}'] = stats[f'STX_CLM_RADIANCE_STDV_PANSHARPENED_{i+1}'].iloc[0]

    eop.save(path=f'{DEIMOS_DIR}/{eop_path}', filesystem=filesystem,
             features=[FeatureType.META_INFO], overwrite_permission=OverwritePermission.OVERWRITE_FEATURES)

In [ ]:
multiprocess(add_new_metadata_radiance, filesystem.listdir(DEIMOS_DIR), max_workers=15)

In [ ]:
multiprocess(add_new_metadata_radiance_pansharpened, filesystem.listdir(DEIMOS_DIR), max_workers=15)

In [ ]:
eop = EOPatch.load(f'{DEIMOS_DIR}/{ filesystem.listdir(DEIMOS_DIR)[0]}', filesystem=filesystem, lazy_loading=True)

In [ ]:
multiprocess(add_new_metadata, filesystem.listdir(DEIMOS_DIR), max_workers=15)

In [ ]:
multiprocess(add_new_metadata_pansharpened, filesystem.listdir(DEIMOS_DIR), max_workers=15)

In [ ]:
DEIMOS_DIR = ''

In [ ]:
multiprocess(add_new_metadata_radiance_pansharpened, filesystem.listdir(DEIMOS_DIR), max_workers=15)

In [ ]:
multiprocess(add_new_metadata_pansharpened, filesystem.listdir(DEIMOS_DIR), max_workers=15)